Methods for loading alerts that are stored locally.


`BasicMultiFilter`: Ampel-alerts/ampel/alert/filter/BasicMultiFilter.py
`FileAlertLoader`: Ampel-alerts/ampel/alert/load/FileAlertLoader.py
`TarAlertLoader`: Ampel-alerts/ampel/alert/load/TarAlertLoader.py

Loaded files are converted to the ampel alert format through the supplier

`ZiAlertSupplier` : ampel.ztf.alert.ZiAlertSupplier
`LSSTAlertSupplier`: ampel.lsst.alert.LSSTAlertSupplier
`ElasticcAlertSupplier`: ampel.lsst.alert.ElasticcAlertSupplier

Forced photometry files can be loaded through 
`ZTFFPbotForcedPhotometryAlertSupplier` : ampel.ztf.alert.ZTFFPbotForcedPhotometryAlertSupplier




In [ ]:
#import os
#from ampel_notebook_utils import * 
from ampel.alert.load.TarAlertLoader import TarAlertLoader
from ampel.alert.load.DirAlertLoader import DirAlertLoader
from ampel.lsst.alert.ElasticcAlertSupplier import ElasticcAlertSupplier
from ampel.lsst.alert.LSSTAlertSupplier import LSSTAlertSupplier
from ampel.ztf.alert.ZTFFPbotForcedPhotometryAlertSupplier import ZTFFPbotForcedPhotometryAlertSupplier 
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel83/Ampel-HU-astro/ampel_conf.yaml'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dumpme',
    purge_db = True,
)

#### Loading alerts stored in a tarfile.


In [ ]:
# ZTF name of transients to explore
tarpath = "/home/jnordin/Downloads/elasticc-test-early-july.tar.gz"

In [ ]:
loader = TarAlertLoader(file_path=tarpath)

In [ ]:
alert = next(loader)

In [ ]:
alert

#### Configuring an alert supplier.

A unit which supplies AMPEL alerts through converting imput from a loader.

In [ ]:
ElasticcAlertSupplier??

In [ ]:
supplier_config =  {
    'deserialize': "avro",
    'loader': {
        'unit': 'TarAlertLoader',
                'config': {'file_path': tarpath}
            }
        }


As a version of the above, we can directly get the latest alert.

In [ ]:
supplier = LSSTAlertSupplier(**supplier_config)

In [ ]:
zs, galcols, mwebvs = [], [], []
for alert in supplier:
    obj_data = [dp for dp in alert.datapoints if 'mwebv' in dp.keys()]
#    if z:= obj_data[0].get('hostgal_zspec', -1)>0:
 #       print('whhhwi z host spec')
  #      print(obj_data)
    if 9 < obj_data[0]['hostgal_mag_u'] < 35 and 9 < obj_data[0]['hostgal_mag_i'] < 35:
        galcols.append( obj_data[0]['hostgal_mag_u'] - obj_data[0]['hostgal_mag_i'] )
        zs.append(obj_data[0]['z_final'])
        mwebvs.append( obj_data[0].get('mwebv') )
        if galcols[-1]> 50:
            print(obj_data)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(zs, galcols, 'o')

So what do we need to do now? Presumably get back to the elasticc test database and extract all dp0, filter for those with mwebv info, then save channel, z, hostgal_u, g, r, i. We then plot the different stuff...

In [ ]:
plt.plot( mwebvs,galcols, 'o')
plt.xlabel('MWEBV')
plt.ylabel('Host color')

Now, reading forced photometry files. We are assuming these are saved locally. 

In [ ]:
ZTFFPbotForcedPhotometryAlertSupplier??

In [ ]:
fpsupplier_config =  {
    'deserialize': None,
    'loader': {
        'unit': 'DirAlertLoader',
                'config': {'folder':'/home/jnordin/data/ztf_fpbot/oct27/full_0_1000', 'extension':'*.csv'}
            }
        }

In [ ]:
fpsupplier = ZTFFPbotForcedPhotometryAlertSupplier(**fpsupplier_config)

In [ ]:
alert = next(fpsupplier)

Let us use this time to repeat how the raw content of an alert (in this case consisting of forced photometry) is converted into a LightCurve object like those which T2 units receive in production.

In [ ]:
from ampel.content.T1Document import T1Document
from ampel.view.LightCurve import LightCurve
from ampel.ztf.ingest.ZiDataPointShaper import ZiDataPointShaper
from ampel.log.AmpelLogger import AmpelLogger
import matplotlib.pyplot as plt

In [ ]:
# A barebones stock document. In an operational DB this would keep track of all the datapoints that were used.
# Here we will use a dummy link.
t1d = T1Document(stock=alert.id, link=0)

In [ ]:
# A shaper is used to convert alert properties (e.g. candid) to persistant AMPEL values (e.g. id)
shaper = ZiDataPointShaper(logger=AmpelLogger.get_logger())

In [ ]:
# Convert all the datapoints
dps = shaper.process( alert.datapoints, alert.id)

In [ ]:
# Construct the lightcurve object
lc = LightCurve.build(t1d, dps)

In [ ]:
# At this point we can e.g. plot the forced photometry 
plt.plot(lc.get_values('jd'), lc.get_values('ampl'), 'o' )